In [8]:
import pandas as pd
import time
import os
import numpy as np
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [9]:
def lgb_modelfit_nocv(params, dtrain, dvalid, predictors, target='target', objective='binary', metrics='auc',
                 feval=None, early_stopping_rounds=20, num_boost_round=3000, verbose_eval=10, categorical_features=None):
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': objective,
        'metric':metrics,
        'learning_rate': 0.001, #to get better accuracy
        'num_leaves': 29,  # we should let it be smaller than 2^(max_depth). Low value to avoid overfitting
        'max_depth': 10,  # -1 means no limit
        'feature_fraction': 0.8, #80% of parameters randomly durting each ietration 
        'min_data_in_leaf': 20,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 255,  # Number of bucketed bin for feature values
        'subsample': 0.6,  # Subsample ratio of the training instance.
        'subsample_freq': 0,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.3,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 5,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 0,  # L1 regularization term on weights
        'reg_lambda': 0,  # L2 regularization term on weights
        'nthread': 4,
        'verbose': 0,
        'metric':metrics
    }

    lgb_params.update(params)

    print("preparing validation datasets")

    xgtrain = lgb.Dataset(dtrain[predictors].values, label=dtrain[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )
    xgvalid = lgb.Dataset(dvalid[predictors].values, label=dvalid[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )

    evals_results = {}

    bst1 = lgb.train(lgb_params, 
                     xgtrain, 
                     valid_sets=[xgtrain, xgvalid], 
                     valid_names=['train','valid'], 
                     evals_result=evals_results, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10, 
                     feval=feval)

    n_estimators = bst1.best_iteration
    print("\nModel Report")
    print("n_estimators : ", n_estimators)
    print(metrics+":", evals_results['valid'][metrics][n_estimators-1])

    return bst1

In [10]:
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [18]:
print('loading train data...')
train_df = pd.read_csv("train.csv", skiprows=range(1,144903891), nrows=10000000, dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])

print('loading test data...')
test_df = pd.read_csv("test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

len_train = len(train_df)
train_df=train_df.append(test_df)



loading train data...
loading test data...


C:\Users\namei\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [19]:
del test_df
gc.collect()

287

In [22]:
print('Extracting new features from the click_time...')
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
gc.collect()

print('grouping by ip-day-hour combination...')
gp_ip_day_hour= train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_tcount'})
train_df = train_df.merge(gp_ip_day_hour, on=['ip','day','hour'], how='left')
gc.collect()

print('grouping by ip-app combination...')
gp_ip_app = train_df[['ip', 'app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
train_df = train_df.merge(gp_ip_app, on=['ip','app'], how='left')
gc.collect()


print('grouping by ip-app-os combination...')
gp_ip_app_os = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
train_df = train_df.merge(gp_ip_app_os, on=['ip','app', 'os'], how='left')
gc.collect()


# Adding features with var and mean hour to make meaning assumptions
print('grouping by : ip_day_chl_var_hour')
gp_ip_day_chl_var_hour = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_tchan_count'})
train_df = train_df.merge(gp_ip_day_chl_var_hour, on=['ip','day','channel'], how='left')
gc.collect()

print('grouping by : ip_app_os_var_hour')
gp_ip_app_os_var_hour = train_df[['ip','app', 'os', 'hour']].groupby(by=['ip', 'app', 'os'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_os_var'})
train_df = train_df.merge(gp_ip_app_os_var_hour, on=['ip','app', 'os'], how='left')
gc.collect()

print('grouping by : ip_app_channel_var_day')
gp_ip_app_channel_var_day = train_df[['ip','app', 'channel', 'day']].groupby(by=['ip', 'app', 'channel'])[['day']].var().reset_index().rename(index=str, columns={'day': 'ip_app_channel_var_day'})
train_df = train_df.merge(gp_ip_app_channel_var_day, on=['ip','app', 'channel'], how='left')
gc.collect()

print('grouping by : ip_app_chl_mean_hour')
gp_ip_app_chl_mean_hour = train_df[['ip','app', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
print("merging...")
train_df = train_df.merge(gp_ip_app_chl_mean_hour, on=['ip','app', 'channel'], how='left')
gc.collect()

Extracting new features from the click_time...
grouping by ip-day-hour combination...
grouping by ip-app combination...
grouping by ip-app-os combination...
grouping by : ip_day_chl_var_hour
grouping by : ip_app_os_var_hour
grouping by : ip_app_channel_var_day
grouping by : ip_app_chl_mean_hour
merging...


56

In [23]:
#deleting all the grouping variables to make space for the model to run and they are not used anywhere else

del gp_ip_day_hour
del gp_ip_app
del gp_ip_app_os
del gp_ip_day_chl_var_hour
del gp_ip_app_os_var_hour
del gp_ip_app_channel_var_day
del gp_ip_app_chl_mean_hour

In [24]:
print("vars and data type: ")
train_df.info()
train_df['ip_tcount'] = train_df['ip_tcount'].astype('uint16')
train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')


test_df = train_df[len_train:]
val_df = train_df[(len_train-2500000):len_train]
train_df = train_df[:(len_train-2500000)]

print("train size: ", len(train_df))
print("valid size: ", len(val_df))
print("test size : ", len(test_df))

target = 'is_attributed'
for col in df.target:
    if np.issubdtype(df[col], np.number):
        df[col].values[:] = 0
        
predictors = ['app','device','os', 'channel', 'hour', 'day', 
              'ip_tcount', 'ip_tchan_count', 'ip_app_count',
              'ip_app_os_count', 'ip_app_os_var',
              'ip_app_channel_var_day','ip_app_channel_mean_hour']
categorical = ['app', 'device', 'os', 'channel', 'hour', 'day']

sub = pd.DataFrame()
sub['click_id'] = test_df['click_id'].astype('int')

gc.collect()

print("Training...")
start_time = time.time()


params = {
    'learning_rate': 0.15,
    'num_leaves': 7,  # 2^max_depth - 1
    'max_depth': 3,  # -1 means no limit
    'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
    'max_bin': 100,  # Number of bucketed bin for feature values
    'subsample': 0.7,  # Subsample ratio of the training instance.
    'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
    'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
    'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
    'scale_pos_weight':99 # because training data is extremely unbalanced 
}
bst = lgb_modelfit_nocv(params, 
                        train_df, 
                        val_df, 
                        predictors, 
                        target, 
                        objective='binary', 
                        metrics='auc',
                        early_stopping_rounds=30, 
                        verbose_eval=True, 
                        num_boost_round=500, 
                        categorical_features=categorical)

print('[{}]: model training time'.format(time.time() - start_time))

vars and data type: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28790469 entries, 0 to 28790468
Data columns (total 17 columns):
app                         uint16
channel                     uint16
click_id                    float64
click_time                  object
device                      uint16
ip                          uint32
is_attributed               float64
os                          uint16
hour                        uint8
day                         uint8
ip_tcount                   int64
ip_app_count                int64
ip_app_os_count             int64
ip_tchan_count              float64
ip_app_os_var               float64
ip_app_channel_var_day      float64
ip_app_channel_mean_hour    float64
dtypes: float64(6), int64(3), object(1), uint16(4), uint32(1), uint8(2)
memory usage: 2.7+ GB
train size:  7500000
valid size:  2500000
test size :  18790469
Training...
preparing validation datasets


C:\Users\namei\Anaconda3\lib\site-packages\lightgbm\basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\namei\Anaconda3\lib\site-packages\lightgbm\basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 30 rounds.
[10]	train's auc: 0.931631	valid's auc: 0.926806
[20]	train's auc: 0.947361	valid's auc: 0.939949
[30]	train's auc: 0.945415	valid's auc: 0.941656
[40]	train's auc: 0.948785	valid's auc: 0.943991
[50]	train's auc: 0.947017	valid's auc: 0.9431
[60]	train's auc: 0.946741	valid's auc: 0.942827
Early stopping, best iteration is:
[35]	train's auc: 0.948186	valid's auc: 0.944058

Model Report
n_estimators :  35
auc: 0.9440577885531274
[64.78816556930542]: model training time


In [25]:
train_df.head()

,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,ip_tcount,ip_app_count,ip_app_os_count,ip_tchan_count,ip_app_os_var,ip_app_channel_var_day,ip_app_channel_mean_hour
0,15,111,NaN,2017-11-09 04:03:08,1,33924,0.0,19,4,9,221,104,25,NaN,22.456667,0.250000,8.000000
1,3,280,NaN,2017-11-09 04:03:08,1,37383,0.0,13,4,9,250,186,50,0.663885,11.795918,0.000000,4.918605
2,15,245,NaN,2017-11-09 04:03:08,1,122294,0.0,10,4,9,179,27,6,0.214286,0.000000,0.000000,4.250000
3,9,145,NaN,2017-11-09 04:03:08,1,73258,0.0,25,4,9,56,67,2,NaN,0.000000,NaN,4.000000
4,15,430,NaN,2017-11-09 04:03:08,1,73347,0.0,13,4,9,42,42,5,NaN,17.200000,0.333333,6.000000


In [14]:
del train_df
del val_df
gc.collect()

print("Predicting...")
sub['is_attributed'] = bst.predict(test_df[predictors])
print("writing...")
sub.to_csv('sub_lgb_balanced99.csv',index=False)
print("done...")

vars and data type: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28790469 entries, 0 to 28790468
Data columns (total 17 columns):
app                         uint16
channel                     uint16
click_id                    float64
click_time                  object
device                      uint16
ip                          uint32
is_attributed               float64
os                          uint16
hour                        uint8
day                         uint8
ip_tcount                   int64
ip_app_count                int64
ip_app_os_count             int64
ip_tchan_count              float64
ip_app_os_var               float64
ip_app_channel_var_day      float64
ip_app_channel_mean_hour    float64
dtypes: float64(6), int64(3), object(1), uint16(4), uint32(1), uint8(2)
memory usage: 2.7+ GB
train size:  7500000
valid size:  2500000
test size :  18790469
Training...
preparing validation datasets


C:\Users\namei\Anaconda3\lib\site-packages\lightgbm\basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\namei\Anaconda3\lib\site-packages\lightgbm\basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 30 rounds.
[10]	train's auc: 0.931631	valid's auc: 0.926806
[20]	train's auc: 0.947361	valid's auc: 0.939949
[30]	train's auc: 0.945415	valid's auc: 0.941656
[40]	train's auc: 0.948785	valid's auc: 0.943991
[50]	train's auc: 0.947017	valid's auc: 0.9431
[60]	train's auc: 0.946741	valid's auc: 0.942827
Early stopping, best iteration is:
[35]	train's auc: 0.948186	valid's auc: 0.944058

Model Report
n_estimators :  35
auc: 0.9440577885531274
[104.46575999259949]: model training time
Predicting...
writing...
done...
